<a href="https://colab.research.google.com/github/VladimirBek/MIFI_tasks/blob/main/%D0%91%D0%B5%D0%BA_%D0%92_%D0%AD_%D0%94%D0%97_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U kaggle_environments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Оставим агента из задания

In [ ]:
%%writefile copy_opponent.py
import random
def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Агент который будет выбирать то, что било оппонента на прошлом ходу

In [ ]:
%%writefile last_opponent_weakness.py

import random


def last_opponent_weakness(observation, configuration):
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
          return 1
        elif observation.lastOpponentAction == 1:
          return 2
        elif observation.lastOpponentAction == 2:
          return 0
    else:
        return random.randrange(0, configuration.signs)

Writing last_opponent_weakness.py


Агент, который будет использовать то, что проиграло оппоненту на прошлом ходу

In [ ]:
%%writefile last_opponent_strength.py

import random

def last_opponent_strength(observation, configuration):
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 2
        elif observation.lastOpponentAction == 1:
            return 0
        elif observation.lastOpponentAction == 2:
            return 1
    else:
        return random.randrange(0, configuration.signs)

Writing last_opponent_strength.py


  Стратегия, которая случайным образом выбирает один из трех вариантов: камень, бумага или ножницы.

In [ ]:
%%writefile random_strategy.py

import random

def random_strategy(observation, configuration):
    return random.randrange(0, configuration.signs)

Writing random_strategy.py


Стратегия, которая циклически выбирает варианты: камень, бумага, ножницы.

In [ ]:
%%writefile cyclic_strategy.py

def cyclic_strategy(observation, configuration):
    return observation.step % configuration.signs

Writing cyclic_strategy.py


Стратегия, которая случайным образом выбирает варианты, но с разными весами для вероятности. Веса статичны.


In [ ]:
%%writefile weighted_random_strategy.py
import random


def weighted_random_strategy(observation, configuration):
    weights = [0.34, 0.33, 0.33]  # Веса для камня, бумаги и ножниц
    return random.choices([0, 1, 2], weights=weights)[0]

Writing weighted_random_strategy.py


На четных ходах вызваются ножницы, а на нечетных камень


In [ ]:
%%writefile even_and_odd.py

import random

def even_and_odd(observation, configuration):
    if observation.step % 2 == 0:
        return 1
    else:
        return 0

Overwriting even_and_odd.py


Стратегия, которая случайным образом выбирает варианты, но с разными весами для вероятности. Веса подбиратся динамически.

In [ ]:
%%writefile  weighted_random_strategy_dinamic.py

import random

def weighted_random_strategy_dinamic(observation, configuration):
    weights = [random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]  # Веса для камня, бумаги и ножниц
    return random.choices([0, 1, 2], weights=weights)[0]

Writing weighted_random_strategy_dinamic.py


Стратегия, которая циклически выбирает варианты, но с небольшой вероятностью случайно меняет выбор.

In [ ]:
%%writefile  cyclic_random_strategy.py
import random

def cyclic_random_strategy(observation, configuration):
    if random.random() < 0.1:
        return random.randrange(0, configuration.signs)
    else:
        return observation.step % configuration.signs

Overwriting cyclic_random_strategy.py


 Стратегия, которая выбирает вариант, который побеждает последний ход противника, но с небольшой вероятностью случайно меняет выбор.

In [ ]:
%%writefile  opposite_random_strategy.py
import random


def opposite_random_strategy(observation, configuration):

    if observation.step > 0:
        if random.random() < 0.1:
            return random.randrange(0, configuration.signs)
        else:
            return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Overwriting opposite_random_strategy.py


На четных ходах вызваются камень, а на нечетных бумага и с небольшой вероятностью могут выпасть ножницы

In [ ]:
%%writefile even_and_odd2.py

import random

def even_and_odd2(observation, configuration):
    if random.random() < 0.1:
        return 1
    if observation.step % 2 == 0:
        return 0
    else:
        return 2

Overwriting even_and_odd2.py


Запускаем турнир, перебирая всех написанных агентов, указывая их в списке

In [ ]:
agents = ["rock",
          "paper",
          "scissors",
          "copy_opponent.py",
          'last_opponent_weakness.py',
          "last_opponent_strength.py",
          "random_strategy.py",
          "cyclic_strategy.py",
          "weighted_random_strategy.py",
          "even_and_odd.py",
          "weighted_random_strategy_dinamic.py",
          "cyclic_random_strategy.py",
          "opposite_random_strategy.py",
          "even_and_odd2.py"
          ]

# Конфигурация игры
configuration = {"episodeSteps": 100}

# Функция для проведения турнира
def run_tournament(agents, configuration):
    results = []
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            agent1 = agents[i]
            agent2 = agents[j]
            match_result = evaluate("rps", [agent1, agent2], configuration)
            results.append((agent1, agent2, match_result))
    return results

# Запуск турнира
tournament_results = run_tournament(agents, configuration)

# Вывод результатов
for result in tournament_results:
    agent1, agent2, match_result = result
    print(f"{agent1} vs {agent2}: {match_result}")

rock vs paper: [[-99.0, 99.0]]
rock vs scissors: [[99.0, -99.0]]
rock vs copy_opponent.py: [[0, 0]]
rock vs last_opponent_weakness.py: [[-99.0, 99.0]]
rock vs last_opponent_strength.py: [[99.0, -99.0]]
rock vs random_strategy.py: [[0, 0]]
rock vs cyclic_strategy.py: [[0, 0]]
rock vs weighted_random_strategy.py: [[0, 0]]
rock vs even_and_odd.py: [[-50.0, 50.0]]
rock vs weighted_random_strategy_dinamic.py: [[0, 0]]
rock vs cyclic_random_strategy.py: [[0, 0]]
rock vs opposite_random_strategy.py: [[-85.0, 85.0]]
rock vs even_and_odd2.py: [[34.0, -34.0]]
paper vs scissors: [[-99.0, 99.0]]
paper vs copy_opponent.py: [[0, 0]]
paper vs last_opponent_weakness.py: [[-97.0, 97.0]]
paper vs last_opponent_strength.py: [[97.0, -97.0]]
paper vs random_strategy.py: [[0, 0]]
paper vs cyclic_strategy.py: [[0, 0]]
paper vs weighted_random_strategy.py: [[0, 0]]
paper vs even_and_odd.py: [[49.0, -49.0]]
paper vs weighted_random_strategy_dinamic.py: [[0, 0]]
paper vs cyclic_random_strategy.py: [[0, 0]]
pape